<a href="https://colab.research.google.com/github/HKang42/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/Harrison_Kang_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

#  Engineer at least two new features.

 - I don't want to split first. Otherwise the features I engineer won't show up in the split data frames

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Let's create a category for any apartment that allows a cat OR a dog

df['catordog'] = ((df['cats_allowed'] == 1 ) | (df['dogs_allowed'] == 1) )

In [0]:
# convert catordog to a numerical column
df['catordog'] = df['catordog'].replace({True:1, False:0})

In [0]:
# let's make a feature for bathrooms per bedroom
# first let's set any 0 values to 0.5

df['bath'] = df['bathrooms'].astype(float).replace( { 0 : 0.5 } )
df['bed'] = df['bedrooms'].astype(float).replace( { 0 : 0.5 } )


In [0]:
df[ 'bathperbed' ] = df['bath'] / df['bed']

#  Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.

In [0]:
train = df [ (df['created'] > '2016-04-01') & (df['created'] < '2016-06-01') ]
test = df [ (df['created'] < '2016-04-01') | (df['created'] > '2016-06-01') ]

#  Fit a linear regression model with at least two features.

In [9]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

x = ['catordog' , 'bathperbed']

y = ['price']

model.fit( train[x], train[y] )

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

#  Get the model's coefficients and intercept.

In [10]:
catdog_coeff = model.coef_[0][0]
bathbed_coeff = model.coef_[0][1]

intercept = model.intercept_[0]

print('Cat or Dog coefficient is:', catdog_coeff)
print('Bath per bed coefficient is:', bathbed_coeff)

print('The y intercept is:', intercept)

Cat or Dog coefficient is: 192.3738354665761
Bath per bed coefficient is: -833.3307765906281
The y intercept is: 4313.3818749469665


#  Get regression metrics RMSE, MAE, and  R2 , for both the train and test data.

In [0]:
# now let's make our predictions for whatever we set our features equal to

predict = model.predict( train[x] )

In [12]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

mae = mean_absolute_error(train['price'], predict)

mse = mean_squared_error(train['price'], predict)
rmse = np.sqrt(mse)

r2 = r2_score(train['price'], predict)

print('For our training data:')
print('Our mean absolute error is: $' + str(mae.round(2)))
print('Our root mean square error is: $' + str(rmse.round(2)))
print('Our r^2 value is: ' + str(r2.round(4)))

For our training data:
Our mean absolute error is: $1109.56
Our root mean square error is: $1697.37
Our r^2 value is: 0.0721


In [0]:
# now let's see how our model looks for the test data

predict = model.predict( test[x] )

In [14]:
mae = mean_absolute_error(test['price'], predict)

mse = mean_squared_error(test['price'], predict)
rmse = np.sqrt(mse)

r2 = r2_score(test['price'], predict)

print('For our test data:')
print('Our mean absolute error is: $' + str(mae.round(2)))
print('Our root mean square error is: $' + str(rmse.round(2)))
print('Our r^2 value is: ' + str(r2.round(4)))

For our test data:
Our mean absolute error is: $1120.34
Our root mean square error is: $1706.34
Our r^2 value is: 0.0632


I looks like the training and testing error is roughly the same which is a decent result. Let's get descriptive statistics for the price and see how big the error is.

In [15]:
df['price'].describe()

count    48817.000000
mean      3579.585247
std       1762.430772
min       1375.000000
25%       2500.000000
50%       3150.000000
75%       4095.000000
max      15500.000000
Name: price, dtype: float64

Given that the mean price is about $3,600, our errors for both data sets are pretty big. Let's try again but with different features.

In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

x = ['bedrooms' , 'balcony']

y = ['price']

model.fit( train[x], train[y] )

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
bedroom_coeff = model.coef_[0][0]
balcony_coeff = model.coef_[0][1]

intercept = model.intercept_[0]

print('Cat or Dog coefficient is:', bedroom_coeff)
print('Bath per bed coefficient is:', balcony_coeff)

print('The y intercept is:', intercept)

Cat or Dog coefficient is: 843.1184113381274
Bath per bed coefficient is: 581.6492956973746
The y intercept is: 2251.6924475790083


In [0]:
predict = model.predict( train[x] )

In [19]:
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

mae = mean_absolute_error(train['price'], predict)

mse = mean_squared_error(train['price'], predict)
rmse = np.sqrt(mse)

r2 = r2_score(train['price'], predict)

print('For our training data:')
print('Our mean absolute error is: $' + str(mae.round(2)))
print('Our root mean square error is: $' + str(rmse.round(2)))
print('Our r^2 value is: ' + str(r2.round(4)))

For our training data:
Our mean absolute error is: $967.05
Our root mean square error is: $1480.6
Our r^2 value is: 0.294


In [0]:
# now let's see how our model looks for the test data

predict = model.predict( test[x] )

In [21]:
mae = mean_absolute_error(test['price'], predict)

mse = mean_squared_error(test['price'], predict)
rmse = np.sqrt(mse)

r2 = r2_score(test['price'], predict)

print('For our test data:')
print('Our mean absolute error is: $' + str(mae.round(2)))
print('Our root mean square error is: $' + str(rmse.round(2)))
print('Our r^2 value is: ' + str(r2.round(4)))

For our test data:
Our mean absolute error is: $981.22
Our root mean square error is: $1480.45
Our r^2 value is: 0.2948


It looks like bedrooms and balconies are a slightly better pair of features based on the error values.